## Homework

In [1]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2021-04-11 20:13:35--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 142.250.65.78, 2607:f8b0:4004:832::200e
Connecting to docs.google.com (docs.google.com)|142.250.65.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4kob34rn87q89363eprotjq184fo7528/1618171950000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2021-04-11 20:13:37--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4kob34rn87q89363eprotjq184fo7528/1618171950000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 172.217.2.97, 2607:f8b0:4004:80a::2001
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleuse

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_theme()

In [3]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [4]:
df.head(5)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,...,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,OwnerOccupier,Nagornoe,5.293465e+06,77878,0.023464,0.195781,4713,2279.0,4,5212,10027.0,8,1,NaN,3,0,2,2,no,0,2,6,no,no,no,no,no,no,no,...,705.32,1186.17,945.74,6,26,32,25,10,1,0,4,12,1,4,31,8,15.19,19.83,92,3478168,49,2235425,348,755.99,1263.47,1009.73,14,85,113,91,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,OwnerOccupier,Poselenie Sosenskoe,6.677245e+07,9553,0.336177,0.072158,656,NaN,0,629,NaN,0,0,NaN,0,0,1,0,no,0,0,1,no,no,no,no,no,yes,no,...,1750.00,2875.00,2312.50,0,0,1,0,1,2,0,0,4,0,0,1,0,34.01,8.70,1,85159,1,189076,14,1108.33,1833.33,1470.83,2,1,4,3,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,Investment,Krylatskoe,1.216448e+07,78507,0.297166,0.000000,4611,3092.0,7,5067,7478.0,7,0,NaN,2,0,14,1,no,0,6,4,no,no,no,no,no,no,no,...,987.50,1645.83,1316.67,7,8,18,9,7,5,1,2,8,0,0,17,0,35.23,4.58,19,755824,19,1015107,95,923.26,1523.26,1223.26,9,18,25,23,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Matushkino,4.708040e+06,38075,0.271702,0.310199,2448,2080.0,3,2748,3885.0,4,0,NaN,0,0,0,0,no,0,0,1,no,no,no,no,no,no,no,...,500.00,850.00,675.00,0,5,3,2,0,0,0,1,2,0,1,4,1,20.84,7.28,3,51038,7,49700,14,507.14,857.14,682.14,0,7,4,3,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,Investment,Gol'janovo,1.428699e+07,157010,0.389354,0.194489,7751,5041.0,6,8004,11081.0,7,0,125.0,3,0,5,3,no,0,5,3,no,no,no,yes,no,no,no,...,608.33,1020.83,814.58,2,7,9,8,0,0,0,3,3,0,0,13,2,36.40,9.08,11,300476,16,509176,54,656.00,1110.00,883.00,4,16,18,12,3,1,0,5,8,0,1,34,5,7900000


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [5]:
from sklearn.model_selection import train_test_split

df_ohe = pd.get_dummies(df) # сделаем копию датасета с закодированными кат. признаками
X_train, X_test, y_train, y_test = train_test_split(df_ohe.drop(['price'], axis=1), df_ohe['price'], random_state=42)

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [6]:
X_train = X_train.fillna(X_train.mean()) # заполним пропущенные значения
X_test = X_test.fillna(X_test.mean())

X_train = X_train.drop(['id', 'timestamp'], axis=1) # уберем id (бесполезен) и timestamp (позднее преобразуем его)
X_test = X_test.drop(['id', 'timestamp'], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  


In [7]:
X_train.shape, X_test.shape

((15000, 451), (5000, 451))

In [8]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(random_state=42)
reg.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

Проверьте качество на отложенной выборке.

In [9]:
from sklearn.metrics import mean_squared_error

y_pred = reg.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

3904446.6366896927

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [10]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [11]:
df['month'] = df['timestamp'].dt.month
df['week_day'] = df['timestamp'].dt.dayofweek
df['rel_floor_max'] = np.nan
df['rel_floor_max'][df['max_floor'] != 0] = df['floor'][df['max_floor'] != 0] / df['max_floor'][df['max_floor'] != 0]
df['rel_kit_flat'] = np.nan
df['rel_kit_flat'][df['full_sq'] != 0] = df['kitch_sq'][df['full_sq'] != 0] / df['full_sq'][df['full_sq'] != 0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [12]:
df_ohe = pd.get_dummies(df) # сделаем 2 версии датасета: обычную и с закодированными кат. признаками, делить будем оба
X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(df_ohe.drop(['price'], axis=1), df_ohe['price'], random_state=42)
X_train, X_test, y_train, y_test = train_test_split(df.drop(['price'], axis=1), df['price'], random_state=42)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [13]:
X_train = X_train.drop(['id', 'timestamp'], axis=1) # заполним пропущенные значения
X_test = X_test.drop(['id', 'timestamp'], axis=1)
X_train_ohe = X_train_ohe.drop(['id', 'timestamp'], axis=1)
X_test_ohe = X_test_ohe.drop(['id', 'timestamp'], axis=1)

X_train = X_train.fillna(X_train.mean()) # уберем id (бесполезен) и timestamp (уже получили новые признаки)
X_test = X_test.fillna(X_test.mean())
X_train_ohe = X_train_ohe.fillna(X_train_ohe.mean())
X_test_ohe = X_test_ohe.fillna(X_test_ohe.mean())

In [14]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

DecisionTreeRegressor.

In [15]:
dtr = DecisionTreeRegressor(random_state=42)
dtr.fit(X_train_ohe, y_train_ohe)
y_pred = dtr.predict(X_test_ohe)
mean_squared_error(y_test, y_pred, squared=False)

3817814.6413202197

RandomForestRegressor.

In [ ]:
rfr = RandomForestRegressor(random_state=42)
rfr.fit(X_train_ohe, y_train_ohe)
y_pred = rfr.predict(X_test_ohe)
mean_squared_error(y_test, y_pred, squared=False)

XGBRegressor.

In [ ]:
xgb = XGBRegressor(random_state=42)
params = {
    'n_estimators': [50, 100, 200]
}
model = GridSearchCV(xgb, param_grid=params)
model.fit(X_train_ohe, y_train_ohe)

In [20]:
model.best_params_, model.best_score_

({'n_estimators': 200}, 0.6895312559295845)

In [21]:
xgb = XGBRegressor(n_estimators=200, random_state=42)
xgb.fit(X_train_ohe, y_train_ohe)
y_pred = xgb.predict(X_test_ohe)
mean_squared_error(y_test, y_pred, squared=False)

[20:22:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


2639734.220688582

LGBMRegressor.

In [25]:
lgb = LGBMRegressor(random_state=42)
params = {
    'n_estimators': [50, 100, 200]
}
model = GridSearchCV(lgb, param_grid=params, cv=5)
model.fit(X_train_ohe, y_train_ohe)

GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=42,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'n_estimators': [50, 100, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [26]:
model.best_params_, model.best_score_

({'n_estimators': 100}, 0.6783636232372183)

In [27]:
lgb = LGBMRegressor(n_estimators=100, random_state=42)
lgb.fit(X_train_ohe, y_train_ohe)
y_pred = lgb.predict(X_test_ohe)
mean_squared_error(y_test, y_pred, squared=False)

2556040.154380731

CatBoostRegressor.

In [28]:
!pip install catboost # установим catboost

     |████████████████████████████████| 67.3MB 59kB/s 


In [29]:
cat_features = np.array(X_train.select_dtypes(include=["object_"]).columns)
cat_features # catboost сам работает с кат. признаками

array(['product_type', 'sub_area', 'culture_objects_top_25',
       'thermal_power_plant_raion', 'incineration_raion',
       'oil_chemistry_raion', 'radiation_raion',
       'railroad_terminal_raion', 'big_market_raion',
       'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line',
       'big_road1_1line', 'railroad_1line', 'ecology'], dtype=object)

In [34]:
X = pd.concat([X_train, X_test], axis=0) # для использования grid_search
y = pd.concat([y_train, y_test], axis=0)
X.shape, y.shape

((20000, 295), (20000,))

In [ ]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(cat_features=cat_features, random_state=42, verbose=False)
params = {
    'depth': [2, 4, 6],
    'iterations': [1000, 1200, 2000]
}
results = cbr.grid_search(param_grid=params, X=X, y=y, partition_random_seed=42, cv=5)

In [ ]:
results['params'] # iterations=1200, depth=6

In [35]:
cbr = CatBoostRegressor(iterations=1200, depth=6, cat_features=cat_features, random_state=42, verbose=False)
cbr.fit(X_train, y_train)
y_pred = cbr.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

2486036.6407551835

Вывод:

Лучший результат показала модель CatBoostRegressor с параметрами `iterations=1200, depth=6`.

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [32]:
invest_train_X = X_train[X_train['product_type'] == 'Investment']
owner_train_X = X_train[X_train['product_type'] == 'OwnerOccupier']

invest_train_y = y_train[X_train['product_type'] == 'Investment']
owner_train_y = y_train[X_train['product_type'] == 'OwnerOccupier']

invest_test_X = X_test[X_test['product_type'] == 'Investment']
owner_test_X = X_test[X_test['product_type'] == 'OwnerOccupier']

invest_test_y = y_test[X_test['product_type'] == 'Investment']
owner_test_y = y_test[X_test['product_type'] == 'OwnerOccupier']

Так как наилучшие результаты показала модель CatBoostRegressor с параметрами `iterations=1200, depth=6`, то будем использовать ее.

In [36]:
cbr_invest = CatBoostRegressor(iterations=1200, random_state=42, verbose=False)
cbr_invest.fit(invest_train_X, invest_train_y, cat_features=cat_features)
invest_pred_y = cbr_invest.predict(invest_test_X)
mean_squared_error(invest_test_y, invest_pred_y, squared=False)

2905136.371026496

In [37]:
cbr_owner = CatBoostRegressor(iterations=1200, random_state=42, verbose=False)
cbr_owner.fit(owner_train_X, owner_train_y, cat_features=cat_features)
owner_pred_y = cbr_owner.predict(owner_test_X)
mean_squared_error(owner_test_y, owner_pred_y, squared=False)

1236195.081772017

In [38]:
y_pred = np.copy(y_test)
y_pred[X_test['product_type'] == 'Investment'] = invest_pred_y
y_pred[X_test['product_type'] == 'OwnerOccupier'] = owner_pred_y
mean_squared_error(y_test, y_pred, squared=False)

2444689.9749587704

Вывод;

С помощью простейшего ансамблирования 2 моделей удалось немного снизить лосс (~40k).

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [39]:
cbr_invest = CatBoostRegressor(iterations=1200, random_state=42, verbose=False)
cbr_invest.fit(invest_train_X, invest_train_y, cat_features=cat_features)
invest_pred_y_cbr = cbr_invest.predict(invest_test_X)
mean_squared_error(invest_test_y, invest_pred_y_cbr, squared=False)

2905136.371026496

In [40]:
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

invest_train_X_ohe = X_train_ohe[X_train['product_type'] == 'Investment']
invest_test_X_ohe = X_test_ohe[X_test['product_type'] == 'Investment']
pipe = make_pipeline(MinMaxScaler(), HuberRegressor(max_iter=500)) #сначала будем нормализовывать данные 
pipe.fit(invest_train_X_ohe, invest_train_y)
invest_pred_y_hr = pipe.predict(invest_test_X_ohe)
mean_squared_error(invest_test_y, invest_pred_y_hr, squared=False)

4173214.1522967364

In [45]:
lst = [] # подберем веса для моделей
for w in np.arange(0.01, 0.1, 0.002):
  invest_pred_y = invest_pred_y_hr * w + invest_pred_y_cbr * (1 - w)
  lst.append((w, mean_squared_error(invest_test_y, invest_pred_y, squared=False)))

In [46]:
sorted(lst, key=lambda x: x[1])[0] # лучший вес, лосс

(0.09, 2890068.589902467)

In [48]:
w = sorted(lst, key=lambda x: x[1])[0][0]
invest_pred_y = invest_pred_y_hr * w + invest_pred_y_cbr * (1 - w)

Итоговый результат.

In [49]:
y_pred = np.copy(y_test)
y_pred[X_test['product_type'] == 'Investment'] = invest_pred_y
y_pred[X_test['product_type'] == 'OwnerOccupier'] = owner_pred_y
mean_squared_error(y_test, y_pred, squared=False)

2433168.5897801598

Вывод:

С помощью ансамблирования tree-based моделей нам удалось добиться значительного понижения итогового лосса.